### **In this notebook, we're going to cover some of the most fundamental concepts of tensors using TensorFlow**

##### *More specifically, we're going to cover:*
#####* Introduction to Tensors
#####* Getting information from Tensors
#####* Manipulating Tensors
#####* Tensors & NumPy
#####* Using @tf.function (a way to speed up your regular Python functions)
#####* Using GPUs with TensorFlow (or TPUs)
#####* Exercises to try for yourself 

### Introduction to Tensors

In [ ]:
# Import TensorFlow
import tensorflow as tf
print(tf.__version__)

2.8.0


In [ ]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [ ]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [ ]:
# Create a vector
vector = tf.constant ([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [ ]:
# Check the dimension of our vector
vector.ndim

1

In [ ]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant ([[10, 7],
                       [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [ ]:
# Check the dimension of our matrix
matrix.ndim

2

In [ ]:
# Create another matrix
another_matrix = tf.constant([[10., 7.],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16) # Specify the data type with dtype parameter

another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [ ]:
# What's the number dimensions of another_matrix?
another_matrix.ndim

2

In [ ]:
# Let's create a tensor
tensor = tf.constant ([[[1, 2, 3],
                        [4, 5, 6]],
                       [[7, 8, 9],
                        [10, 11, 12]],
                       [[13, 14, 15],
                        [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [ ]:
# What's the number dimensions of tensor?
tensor.ndim

3

###**What we've created so far:**

##### * Scalar: a single number
##### * Vector: a number with direction (e.g. wind speed and direction)
##### * Matrix: a 2-dimensional array of numbers
##### * Tensor: an n-dimensional array of numbers (when n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

### Creating tensors with `tf.Variable`

In [ ]:
tf.Variable

tensorflow.python.ops.variables.Variable

In [ ]:
# Create the same tensor with tf.Vairable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [ ]:
# Let's try change one of the elements in our changeable tensor
changeable_tensor[0] = 7
changeable_tensor

TypeError: ignored

In [ ]:
# How about we try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

In [ ]:
# Let's try change our unchangeable tensor
unchangeable_tensor[0].assign(7)
unchangeable_tensor

##### **Note**: Rarely in practice will you need to decide whether to use `tf.constant` or `tf.Variable` to create tensors, as Tensorflow does this for you. However, if in doubt, use `tf.constant` and change it later if needed.

### Creating random tensors

##### Random tensors are tensors of some arbitary size which contain random numbers.

In [ ]:
# Create two random (but the sam) tensors
random_1 = tf.random.Generator.from_seed(7) # set seed for reproducability
random_1 = random_1.normal(shape=(3, 2))
random_2 = tf.random.Generator.from_seed(7)
random_2 = random_2.normal(shape=(3, 2))

# Are they equal
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements in a tensor

In [ ]:
# Shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesn't effect learning)
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])

# Shuffle our non-shuffled tensor
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  4],
       [10,  7],
       [ 2,  5]], dtype=int32)>

In [ ]:
# Shuffle our non-shuffled tensor
tf.random.set_seed(42)
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [ ]:
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

**EXERCISE**

##### ✍︎ **Exercise**: Read through TensorFlow documentation on random seed generation: https://www.tensorflow.org/api_docs/python/tf/random/set_seed and practice writing 5 random tensors and shuffle them.

##### It looks like if we want our shuffled tensors to be in the same order, we've got to use the global level random seed as well as the operation level random seed:

> Rule 4: If both the global and the operation seeds are set; both seeds are used in conjunction to determine the random sequence.

In [ ]:
# Shuffling the first tensor
tens1 = tf.constant([[10,7],
                    [40, 19],
                    [19, 100]]) 
tens1

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 10,   7],
       [ 40,  19],
       [ 19, 100]], dtype=int32)>

In [ ]:
tens1.ndim

2

In [ ]:
tf.random.shuffle(tens1)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 10,   7],
       [ 19, 100],
       [ 40,  19]], dtype=int32)>

In [ ]:
tf.random.set_seed(50)
tf.random.shuffle(tens1, seed=50)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 19, 100],
       [ 40,  19],
       [ 10,   7]], dtype=int32)>

In [ ]:
# Experimenting with seeds
tf.random.set_seed(50)
tf.random.shuffle(tens1, seed=19)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 10,   7],
       [ 40,  19],
       [ 19, 100]], dtype=int32)>

In [ ]:
# Shuffling the second tensor
tens2 = tf.constant([[99, 199, 50],
                     [99, 83, 695],
                     [75, 89, 22],
                     [65, 9999, 20]])
tens2

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[  99,  199,   50],
       [  99,   83,  695],
       [  75,   89,   22],
       [  65, 9999,   20]], dtype=int32)>

In [ ]:
tens2.ndim

2

In [ ]:
tf.random.shuffle(tens2)

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[  65, 9999,   20],
       [  99,  199,   50],
       [  75,   89,   22],
       [  99,   83,  695]], dtype=int32)>

In [ ]:
tf.random.set_seed(99)
tf.random.shuffle(tens2, seed=99)

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[  99,  199,   50],
       [  75,   89,   22],
       [  99,   83,  695],
       [  65, 9999,   20]], dtype=int32)>

In [ ]:
tf.random.shuffle(tens2, seed=1)

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[  75,   89,   22],
       [  99,   83,  695],
       [  99,  199,   50],
       [  65, 9999,   20]], dtype=int32)>

In [ ]:
# Shuffling the third tensor
tens3 = ([[55, 67, 86, 99],
          [86, 96, 90],
          [85, 37, 84, 239, 49],
          [84, 93],
          [65, 90, 82]])
tens3

[[55, 67, 86, 99], [86, 96, 90], [85, 37, 84, 239, 49], [84, 93], [65, 90, 82]]

In [ ]:
tens3.ndim

AttributeError: ignored

In [ ]:
# Shuffling the fourth tensor
tens4 = tf.constant([[89, 80],
                     [90, 99],
                     [969, 6565],
                     [84, 754]])
tens4

<tf.Tensor: shape=(4, 2), dtype=int32, numpy=
array([[  89,   80],
       [  90,   99],
       [ 969, 6565],
       [  84,  754]], dtype=int32)>

In [ ]:
tens4.ndim

2

In [ ]:
tf.random.shuffle(tens4)

<tf.Tensor: shape=(4, 2), dtype=int32, numpy=
array([[  89,   80],
       [  90,   99],
       [  84,  754],
       [ 969, 6565]], dtype=int32)>

In [ ]:
tf.random.set_seed(1092)
tf.random.shuffle(tens4, seed=1092)

<tf.Tensor: shape=(4, 2), dtype=int32, numpy=
array([[  84,  754],
       [  90,   99],
       [  89,   80],
       [ 969, 6565]], dtype=int32)>

In [ ]:
# Experimentation with seed=
tf.random.set_seed(1092)
tf.random.shuffle(tens4, seed=1000)

<tf.Tensor: shape=(4, 2), dtype=int32, numpy=
array([[ 969, 6565],
       [  90,   99],
       [  84,  754],
       [  89,   80]], dtype=int32)>

In [ ]:
# Shuffling the fifth tensor
tens5 = tf.constant([[98, 100, 1000, 10000],
                     [89, 90, 98, 987],
                     [89, 74, 822, 736]])
tens5

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[   98,   100,  1000, 10000],
       [   89,    90,    98,   987],
       [   89,    74,   822,   736]], dtype=int32)>

In [ ]:
tens5.ndim

2

In [ ]:
tf.random.shuffle(tens5)

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[   98,   100,  1000, 10000],
       [   89,    74,   822,   736],
       [   89,    90,    98,   987]], dtype=int32)>

In [ ]:
tf.random.set_seed(9090)
tf.random.shuffle(tens5, seed=93)

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[   98,   100,  1000, 10000],
       [   89,    90,    98,   987],
       [   89,    74,   822,   736]], dtype=int32)>

In [ ]:
tf.random.set_seed(42) # Global level random seed
tf.random.shuffle(not_shuffled, seed=42) # Operational level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

### Other ways to make tensors

In [ ]:
# Create a tensor of all ones
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [ ]:
# Create a tensor of all zeroes
tf.zeros(shape=(3,4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn NumPy arrays into tensors

##### The main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU (much faster for numerical computing)

In [ ]:
# You can also turn NumPy arrays into tensors
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32) # create a NumPy array between 1 and 25
numpy_A

# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector) # non capital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [ ]:
A = tf.constant(numpy_A)
A

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)>

In [ ]:
A = tf.constant(numpy_A, shape=(2, 3, 4))
B = tf.constant(numpy_A)
A, B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

In [ ]:
2 * 3 * 4

24

In [ ]:
A = tf.constant(numpy_A, shape=(2, 3, 5))
B = tf.constant(numpy_A)
A, B

TypeError: ignored

In [ ]:
2 * 3 * 5

30

In [ ]:
A = tf.constant(numpy_A, shape=(2, 2, 3, 2))
B = tf.constant(numpy_A)
A, B

(<tf.Tensor: shape=(2, 2, 3, 2), dtype=int32, numpy=
 array([[[[ 1,  2],
          [ 3,  4],
          [ 5,  6]],
 
         [[ 7,  8],
          [ 9, 10],
          [11, 12]]],
 
 
        [[[13, 14],
          [15, 16],
          [17, 18]],
 
         [[19, 20],
          [21, 22],
          [23, 24]]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

In [ ]:
A = tf.constant(numpy_A, shape=(3, 8))
B = tf.constant(numpy_A)
A, B

(<tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16],
        [17, 18, 19, 20, 21, 22, 23, 24]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

In [ ]:
A.ndim

2

In [ ]:
# Self-attempt
selfap = np.arange(1, 101, dtype=np.int32)
selfap

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100], dtype=int32)

In [ ]:
C = tf.constant(selfap)
C

<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100], dtype=int32)>

In [ ]:
C = tf.constant(selfap, shape=(50, 2))
D = tf.constant(selfap)
C, D

(<tf.Tensor: shape=(50, 2), dtype=int32, numpy=
 array([[  1,   2],
        [  3,   4],
        [  5,   6],
        [  7,   8],
        [  9,  10],
        [ 11,  12],
        [ 13,  14],
        [ 15,  16],
        [ 17,  18],
        [ 19,  20],
        [ 21,  22],
        [ 23,  24],
        [ 25,  26],
        [ 27,  28],
        [ 29,  30],
        [ 31,  32],
        [ 33,  34],
        [ 35,  36],
        [ 37,  38],
        [ 39,  40],
        [ 41,  42],
        [ 43,  44],
        [ 45,  46],
        [ 47,  48],
        [ 49,  50],
        [ 51,  52],
        [ 53,  54],
        [ 55,  56],
        [ 57,  58],
        [ 59,  60],
        [ 61,  62],
        [ 63,  64],
        [ 65,  66],
        [ 67,  68],
        [ 69,  70],
        [ 71,  72],
        [ 73,  74],
        [ 75,  76],
        [ 77,  78],
        [ 79,  80],
        [ 81,  82],
        [ 83,  84],
        [ 85,  86],
        [ 87,  88],
        [ 89,  90],
        [ 91,  92],
        [ 93,  94],
        [ 95

In [ ]:
E = tf.constant(selfap, shape=(2, 2, 5, 5))
E, D

(<tf.Tensor: shape=(2, 2, 5, 5), dtype=int32, numpy=
 array([[[[  1,   2,   3,   4,   5],
          [  6,   7,   8,   9,  10],
          [ 11,  12,  13,  14,  15],
          [ 16,  17,  18,  19,  20],
          [ 21,  22,  23,  24,  25]],
 
         [[ 26,  27,  28,  29,  30],
          [ 31,  32,  33,  34,  35],
          [ 36,  37,  38,  39,  40],
          [ 41,  42,  43,  44,  45],
          [ 46,  47,  48,  49,  50]]],
 
 
        [[[ 51,  52,  53,  54,  55],
          [ 56,  57,  58,  59,  60],
          [ 61,  62,  63,  64,  65],
          [ 66,  67,  68,  69,  70],
          [ 71,  72,  73,  74,  75]],
 
         [[ 76,  77,  78,  79,  80],
          [ 81,  82,  83,  84,  85],
          [ 86,  87,  88,  89,  90],
          [ 91,  92,  93,  94,  95],
          [ 96,  97,  98,  99, 100]]]], dtype=int32)>,
 <tf.Tensor: shape=(100,), dtype=int32, numpy=
 array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,

### Getting information from tensors

##### When dealing with tensors you probably want to be aware of the following attributes:
* Shape
* Rank
* Axis or Dimension
* Size

In [ ]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros([2, 3, 4, 5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [ ]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [ ]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [ ]:
2 * 3 * 4 * 5

120

In [ ]:
# Get various attributes of our tensor
print("Datatype of every element:", rank_4_tensor.dtype)
print("Number of dimensions (rank):", rank_4_tensor.ndim)
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along the 0 axis", rank_4_tensor.shape[0])
print("Elements along the last axis", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor", tf.size(rank_4_tensor).numpy())
print("Total number of elements in our tensor", tf.size(rank_4_tensor))

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along the 0 axis 2
Elements along the last axis 5
Total number of elements in our tensor 120
Total number of elements in our tensor tf.Tensor(120, shape=(), dtype=int32)
